In [1]:
import pandas as pd


In [2]:
path = '../data/raw/ValeursFoncieres-2025-S1.txt'

In [11]:
df = pd.read_csv(
    path,
    sep="|",
    encoding="latin-1",
    low_memory=False)


In [12]:
df.head()


,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/01/2025,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,J,NaN,78.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/01/2025,Vente,...,NaN,0,1.0,Maison,NaN,111.0,5.0,S,NaN,133.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/01/2025,Vente,...,NaN,0,3.0,DÃ©pendance,NaN,0.0,0.0,S,NaN,133.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,06/01/2025,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,S,NaN,46.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,06/01/2025,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,J,NaN,17.0


In [13]:
cols_utiles = [
    "Valeur fonciere",
    "Surface reelle bati",
    "Nombre pieces principales",
    "Code departement",
    "Type local",
    "Code postal"]

In [14]:
df = df[cols_utiles]


In [15]:
for col in ["Valeur fonciere", "Surface reelle bati", "Nombre pieces principales", "Code postal"]:
    df[col] = (df[col].astype(str).str.replace(",",".", regex=False).astype(float))

In [16]:
df = df.dropna()

In [17]:
df.head()

,Valeur fonciere,Surface reelle bati,Nombre pieces principales,Code departement,Type local,Code postal
1,468000.0,111.0,5.0,01,Maison,1550.0
2,468000.0,0.0,0.0,01,DÃ©pendance,1550.0
5,180000.0,87.0,4.0,01,Appartement,1200.0
7,490000.0,0.0,0.0,01,DÃ©pendance,1310.0
8,490000.0,0.0,0.0,01,DÃ©pendance,1310.0


In [15]:
df = df[df["Code departement"] == "75"]

In [16]:
df = df[df["Type local"] == "Appartement" ]

In [17]:
df

,Valeur fonciere,Surface reelle bati,Nombre pieces principales,Code departement,Type local
1355674,430500.0,48.0,3.0,75,Appartement
1355676,527700.0,40.0,1.0,75,Appartement
1355678,383000.0,50.0,2.0,75,Appartement
1355681,199000.0,16.0,1.0,75,Appartement
1355682,199000.0,117.0,5.0,75,Appartement
...,...,...,...,...,...
1387061,835000.0,70.0,4.0,75,Appartement
1387063,375000.0,33.0,1.0,75,Appartement
1387064,1370000.0,92.0,2.0,75,Appartement
1387072,550000.0,61.0,3.0,75,Appartement


In [19]:
df["Code postal"] = df["Code postal"].astype(str)

In [21]:
df["Code postal"].value_counts().head(10)

Code postal
93130.0    1984
6000.0     1869
75018.0    1443
21000.0    1343
75015.0    1335
75017.0    1244
51100.0    1221
75016.0    1139
33000.0    1137
6400.0     1085
Name: count, dtype: int64

In [22]:
df["Code postal"] = ( df["Code postal"].astype(str).str.replace(".0","", regex=False))

In [23]:
df = df[df["Code postal"].str.match(r"^750\d{2}$")]

In [24]:
df["arrondissement"] = df["Code postal"].str[-2:].astype(int)

In [25]:
df["arrondissement"].value_counts().sort_index()

arrondissement
1      212
2      338
3      383
4      324
5      429
6      397
7      669
8      532
9      832
10     850
11     932
12     853
13     657
14     828
15    1335
16    1139
17    1244
18    1443
19     699
20     982
Name: count, dtype: int64

In [26]:
df = df[df["Surface reelle bati"] > 9 ]

In [28]:
df["prix_m2"] = df["Valeur fonciere"] / df["Surface reelle bati"]

In [29]:
df = df[(df["prix_m2"] > 2000) & (df["prix_m2"] < 30000 )]

In [30]:
df[["prix_m2"]].describe()

,prix_m2
count,12339.000000
mean,10537.444808
std,4161.312715
min,2009.259259
25%,8126.749653
50%,9803.921569
75%,11944.444444
max,29965.544944


In [31]:
outpath = "../data/processed/paris_appart_clean_01.csv"

In [33]:
df.to_csv(
    outpath,
    index=False,
    encoding="utf-8")